In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!pip install tqdm ta yfinance xgboost

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 948.2/948.2 kB 16.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.5/108.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.9/153.9 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 201.4/201.4 MB 5.3 MB/s eta 0:00:00
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=dda739452e34c4db00d593718d6784111212cbc935c53416f7bbaea75e73887f
  Stored in directory: /root/.cache/pip/wheels/a1/d7/29/7781cc5eb9a3659d032d7d15bdd0f49d07d2b24fec29f44bc4
  Created wheel for peewee: filename=peewee-3.17.8-cp311-cp311-linux_x86_64.whl size=886571 sh

In [4]:
!pip install scikit-learn==1.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 22.2 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.6.1
    Uninstalling scikit-learn-1.6.1:
      Successfully uninstalled scikit-learn-1.6.1


In [ ]:
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import TimeSeriesSplit, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, precision_score, recall_score, f1_score, roc_auc_score
import xgboost as xgb
import matplotlib.pyplot as plt
from ta import add_all_ta_features
from ta.momentum import RSIIndicator
from sklearn.metrics import confusion_matrix
import seaborn as sns
from IPython.display import display
import warnings
import os

warnings.filterwarnings("ignore", category=RuntimeWarning)

# --- Configuration ---
base_dir = '/content/drive/MyDrive'  # Update this to your desired directory
train_size_ratio = 0.8

# Create required directories
os.makedirs(os.path.join(base_dir, 'results'), exist_ok=True)
os.makedirs(os.path.join(base_dir, 'companies'), exist_ok=True)

# Load tickers
tickers_path = os.path.join(base_dir, 'Tickers.csv')  # Ensure this file exists with a 'Ticker' column
tickers_df = pd.read_csv(tickers_path)
tickers = tickers_df['Ticker'].tolist()

# Initialize progress tracking
ranked_filepath = os.path.join(base_dir, 'companies', 'ranked_companies.csv')
processed_tickers = set()

if os.path.exists(ranked_filepath):
    companies_df = pd.read_csv(ranked_filepath)
    companies_data = companies_df.to_dict('records')
    processed_tickers = set(companies_df['Ticker'])
else:
    companies_data = []

# --- Modified Preprocessing ---
def preprocess_data(df: pd.DataFrame) -> pd.DataFrame:
    print("\n🔧 Preprocessing data...")
    try:
        if isinstance(df.columns, pd.MultiIndex):
            df = df.copy()
            df.columns = df.columns.get_level_values(0)

        df = df.replace([np.inf, -np.inf], np.nan)
        df.ffill(inplace=True)
        df.bfill(inplace=True)

        # Create target variable
        df['Log_Return'] = np.log(df['Close'].shift(-1) / df['Close'])
        df.dropna(subset=['Log_Return'], inplace=True)

        # Log-transform
        price_cols = ['Open', 'High', 'Low', 'Close']
        df[price_cols] = np.log(df[price_cols])
        df['Volume'] = np.log(df['Volume'].replace(0, 1e-5))

        # Stationary features
        for col in price_cols + ['Volume']:
            df[f'd_log_{col}'] = df[col].diff()
            df[f'd_log_{col}'] = df[f'd_log_{col}'].replace([np.inf, -np.inf], np.nan)

        df.dropna(subset=[f'd_log_{col}' for col in price_cols + ['Volume']],
                 how='any', inplace=True)

        # Technical indicators
        rsi = RSIIndicator(df['d_log_Close'], window=14)
        df['RSI'] = rsi.rsi().replace([np.inf, -np.inf], np.nan)

        for window in [5, 20]:
            df[f'Roll_Mean_{window}'] = df['Log_Return'].rolling(window, min_periods=3).mean()
            df[f'Roll_Std_{window}'] = df['Log_Return'].rolling(window, min_periods=3).std()

        # TA features
        df = df.copy()
        df = add_all_ta_features(
            df,
            open="d_log_Open",
            high="d_log_High",
            low="d_log_Low",
            close="d_log_Close",
            volume="d_log_Volume",
            fillna=True
        )

        # Final NA handling
        df.replace([np.inf, -np.inf], np.nan, inplace=True)
        df.dropna(inplace=True, how='any')

        print(f"✅ Preprocessed data shape: {df.shape}")
        return df

    except Exception as e:
        print(f"❌ Preprocessing failed: {str(e)}")
        raise

# --- Evaluation metrics ---
def evaluate_model(model, X, y, dataset_name):
    y_pred = model.predict(X)
    metrics = {
        'MSE': mean_squared_error(y, y_pred),
        'RMSE': np.sqrt(mean_squared_error(y, y_pred)),
        'MAE': mean_absolute_error(y, y_pred),
        'R2': r2_score(y, y_pred),
        'Error/Return Ratio': np.mean(np.abs(y - y_pred)) / np.mean(np.abs(y))
    }
    print(f"\n📈 {dataset_name} Evaluation:")
    for name, value in metrics.items():
        print(f"{name}: {value:.6f}" if isinstance(value, float) else f"{name}: {value}")
    return metrics

# --- Main processing loop ---
for ticker in tickers:
    try:
        if ticker in processed_tickers:
            print(f"Skipping already processed {ticker}")
            continue

        print(f"\n{'='*40}\nProcessing {ticker}\n{'='*40}")

        # --- Data Collection Update ---
        try:
            data = yf.download(ticker, period="max")
            # Remove any rows with invalid prices
            data = data[(data[['Open', 'High', 'Low', 'Close']] > 0).all(axis=1)]
            print(f"✅ Downloaded {len(data)} clean records")
        except Exception as e:
            print(f"❌ Data download failed for {ticker}: {str(e)}")
            continue

        if data.empty:
            print(f"No data available for {ticker}")
            continue

        # --- Preprocessing ---
        try:
            processed_data = preprocess_data(data.copy())
        except Exception as e:
            print(f"❌ Preprocessing failed for {ticker}: {str(e)}")
            continue

        # Split data
        train_size = int(len(processed_data) * train_size_ratio)
        test_size = len(processed_data) - train_size

        train_data = processed_data.iloc[:train_size]
        test_data = processed_data.iloc[train_size:]

        print(f"\n📊 Dataset split:")
        print(f"- Training samples: {len(train_data)}")
        print(f"- Test samples: {len(test_data)}")

        # Prepare features
        excluded_cols = ['Log_Return', 'Close']
        xgb_features = [col for col in processed_data.columns if col not in excluded_cols]

        X_train = train_data[xgb_features]
        y_train = train_data['Log_Return']
        X_test = test_data[xgb_features]
        y_test = test_data['Log_Return']

        # Initial model for evaluation
        eval_model = xgb.XGBRegressor(
        n_estimators=10000,
        learning_rate=0.01,
        max_depth=2,
        subsample=0.8,
        colsample_bytree=0.8,
        random_state=42
        )
        eval_model.fit(X_train, y_train)
        print("\n✅ Evaluation model training completed")

        # Evaluation metrics
        train_metrics = evaluate_model(eval_model, X_train, y_train, "Train")
        test_metrics = evaluate_model(eval_model, X_test, y_test, "Test")

        # Classification metrics
        y_pred_test = eval_model.predict(X_test)
        y_true_binary = np.where(y_test > 0, 1, 0)
        y_pred_binary = np.where(y_pred_test > 0, 1, 0)

        print("\n📊 Classification Metrics:")
        precision = precision_score(y_true_binary, y_pred_binary)
        recall = recall_score(y_true_binary, y_pred_binary)
        f1 = f1_score(y_true_binary, y_pred_binary)
        auc_roc = roc_auc_score(y_true_binary, y_pred_test)
        print(f"Precision: {precision:.4f}")
        print(f"Recall: {recall:.4f}")
        print(f"F1 Score: {f1:.4f}")
        print(f"AUC-ROC: {auc_roc:.4f}")

        # Final model trained on entire dataset
        print("\n🔁 Training final model on full dataset...")
        X_full = processed_data[xgb_features]
        y_full = processed_data['Log_Return']
        final_model = xgb.XGBRegressor(
            n_estimators=2000,
            learning_rate=0.01,
            max_depth=3,
            subsample=0.8,
            colsample_bytree=0.8,
            random_state=42
        ).fit(X_full, y_full)
        print("✅ Full dataset training completed")

        # Final prediction using full model
        last_data_point = processed_data[xgb_features].iloc[[-1]]
        next_day_pred = final_model.predict(last_data_point)
        print(f"\n🔮 Next-period Prediction:")
        print(f"- % Return: {100*next_day_pred[0]:.4f}")
        print(f"- Price Ratio: {np.exp(next_day_pred[0]):.6f} (1 = no change)")

        # --- Save results ---
        safe_ticker = ticker.replace('^', '_').replace('.', '_').replace('/', '_')
        ticker_dir = os.path.join(base_dir, 'results', safe_ticker)
        os.makedirs(ticker_dir, exist_ok=True)

        # Save metrics in the specified format
        metrics_df = pd.DataFrame({
            'Train': train_metrics,
            'Test': test_metrics
        }).T

        # Add classification metrics
        metrics_df['Precision'] = [np.nan, precision]
        metrics_df['Recall'] = [np.nan, recall]
        metrics_df['F1_Score'] = [np.nan, f1]
        metrics_df['AUC_ROC'] = [np.nan, auc_roc]

        metrics_df.to_csv(os.path.join(ticker_dir, 'metrics.csv'))
        print(f"✅ Saved metrics for {ticker}")

        # --- Visualization ---
        plt.figure(figsize=(10, 10))
        sns.regplot(x=y_test, y=y_pred_test, scatter_kws={'alpha':0.3})
        plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], '--r')
        plt.xlabel('Actual Returns')
        plt.ylabel('Predicted Returns')
        plt.title(f'Regression Plot for {ticker}')
        plt.savefig(os.path.join(ticker_dir, 'regression_plot.png'))
        plt.close()
        print(f"✅ Saved regression plot for {ticker}")

        # --- Corrected Confusion Matrix ---
        def create_confusion_matrix(y_true, y_pred):
            y_true_sign = np.where(y_true > 0, 1, 0)
            y_pred_sign = np.where(y_pred > 0, 1, 0)
            return confusion_matrix(y_true_sign, y_pred_sign, labels=[1, 0])

        cm = create_confusion_matrix(y_test, y_pred_test)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                    xticklabels=['Positive', 'Negative'],
                    yticklabels=['Positive', 'Negative'])
        plt.xlabel('Predicted')
        plt.ylabel('Actual')
        plt.title(f'Confusion Matrix for {ticker}')
        plt.savefig(os.path.join(ticker_dir, 'confusion_matrix.png'))
        plt.close()
        print(f"✅ Saved confusion matrix for {ticker}")

        # --- Update rankings ---
        companies_data.append({
            'Ticker': ticker,
            'Next_Predicted_Return': next_day_pred[0],
            'Error/Return Ratio': test_metrics['Error/Return Ratio'],
            'F1_Score': f1
        })

        # Save intermediate results
        pd.DataFrame(companies_data).sort_values(
            by='Next_Predicted_Return', ascending=False
        ).to_csv(ranked_filepath, index=False)

        processed_tickers.add(ticker)
        print(f"Completed processing {ticker}")

    except Exception as e:
        print(f"Error processing {ticker}: {str(e)}")
        continue

print("\nProcessing complete! Final results saved to:")
print(f"- Company results: {os.path.join(base_dir, 'results')}")
print(f"- Rankings: {ranked_filepath}")

Skipping already processed 20MICRONS.NS
Skipping already processed ACCELYA.NS
Skipping already processed ACCENTMIC.NS
Skipping already processed ACE.NS
Skipping already processed ACI.NS
Skipping already processed ADFFOODS.NS
Skipping already processed ADVANIHOTR.NS
Skipping already processed AEGISLOG.NS
Skipping already processed AEROFLEX.NS
Skipping already processed AGARIND.NS
Skipping already processed AGIIL.NS
Skipping already processed AIAENG.NS
Skipping already processed AIIL.NS
Skipping already processed AKZOINDIA.NS
Skipping already processed ALBERTDAVD.NS
Skipping already processed ALKEM.NS
Skipping already processed AMNPLST.NS
Skipping already processed AMRUTANJAN.NS

Processing ANDHRAPAP.NS


[*********************100%***********************]  1 of 1 completed


✅ Downloaded 7301 clean records

🔧 Preprocessing data...


/usr/local/lib/python3.11/dist-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


✅ Preprocessed data shape: (0, 102)

📊 Dataset split:
- Training samples: 0
- Test samples: 0


...

✅ Preprocessed data shape: (4148, 102)

📊 Dataset split:
- Training samples: 3318
- Test samples: 830

✅ Evaluation model training completed

📈 Train Evaluation:
MSE: 0.000047
RMSE: 0.006871
MAE: 0.005366
R2: 0.937313
Error/Return Ratio: 0.288662

📈 Test Evaluation:
MSE: 0.000126
RMSE: 0.011205
MAE: 0.008406
R2: 0.757926
Error/Return Ratio: 0.503620

📊 Classification Metrics:
Precision: 0.8110
Recall: 0.8744
F1 Score: 0.8415
AUC-ROC: 0.9055

🔁 Training final model on full dataset...
✅ Full dataset training completed

🔮 Next-period Prediction:
- % Return: -1.5679
- Price Ratio: 0.984444 (1 = no change)
✅ Saved metrics for NMDC.NS
✅ Saved regression plot for NMDC.NS
✅ Saved confusion matrix for NMDC.NS
Completed processing NMDC.NS

Processing NUCLEUS.NS


[*********************100%***********************]  1 of 1 completed


✅ Downloaded 5481 clean records

🔧 Preprocessing data...


/usr/local/lib/python3.11/dist-packages/ta/trend.py:1030: FutureWarning: Series.__setitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To set a value by position, use `ser.iloc[pos] = value`
  self._psar[i] = high2


✅ Preprocessed data shape: (5466, 102)

📊 Dataset split:
- Training samples: 4372
- Test samples: 1094

✅ Evaluation model training completed

📈 Train Evaluation:
MSE: 0.000088
RMSE: 0.009394
MAE: 0.007277
R2: 0.967940
Error/Return Ratio: 0.317616

📈 Test Evaluation:
MSE: 0.000354
RMSE: 0.018802
MAE: 0.011762
R2: 0.607504
Error/Return Ratio: 0.588936

📊 Classification Metrics:
Precision: 0.8690
Recall: 0.6791
F1 Score: 0.7624
AUC-ROC: 0.8859

🔁 Training final model on full dataset...
